# Object Detection

## Setup

In [ ]:
#@title

import os

!pip install --quiet tensorflow_text
os.environ["TFHUB_MODEL_LOAD_FORMAT"] = "COMPRESSED"

In [ ]:
#@title

import os
import math

import numpy as np
import requests

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as tf_text
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.set_style("whitegrid", {'axes.grid' : False})

In [ ]:
%load_ext tensorboard

In [ ]:
import requests

def download_image(url, path):
    r = requests.get(url, allow_redirects=True)
    with open(path, 'wb') as f:
        f.write(r.content)
    return path

def plot(y, titles=None):
    for i, image in enumerate(y):
        if image is None:
            plt.subplot(1, len(y), i+1)
            plt.axis('off')
            continue
            
        t = titles[i] if titles else None
        plt.subplot(1, len(y), i+1, title=t)
        plt.imshow(image)
        plt.axis('off')
    plt.tight_layout()

## Model Definition

In [ ]:
detector = hub.load("https://tfhub.dev/tensorflow/efficientdet/d4/1")

## Application

In [ ]:
INPUT_SHAPE = [299, 299, 3]

DATA_DIR = 'images/'
IMAGES = [
  'https://raw.githubusercontent.com/keisen/tf-keras-vis/master/examples/images/goldfish.jpg',
  'https://raw.githubusercontent.com/keisen/tf-keras-vis/master/examples/images/bear.jpg',
  'https://raw.githubusercontent.com/keisen/tf-keras-vis/master/examples/images/soldiers.jpg',
  'https://3.bp.blogspot.com/-W__wiaHUjwI/Vt3Grd8df0I/AAAAAAAAA78/7xqUNj8ujtY/s400/image02.png'
]

In [ ]:
#@title


os.makedirs(os.path.join(DATA_DIR, 'unknown'), exist_ok=True)

for i in IMAGES:
    _, f = os.path.split(i)
    download_image(i, os.path.join(DATA_DIR, 'unknown', f))

In [ ]:
images_set = (
    tf.keras.preprocessing.image_dataset_from_directory(
        DATA_DIR,
        image_size=INPUT_SHAPE[:2],
        batch_size=32,
        shuffle=False)
    .cache()
    .prefetch(buffer_size=tf.data.experimental.AUTOTUNE))

In [ ]:
#@title

plt.figure(figsize=(12, 4))
for images, _ in images_set.take(1):
    for i, image in enumerate(images):
        plt.subplot(math.ceil(len(images) / 4), 4, i+1)
        plt.imshow(image.numpy().astype('uint8'))
        plt.axis('off')

plt.tight_layout()

In [ ]:
inputs = tf.cast(images[3:4], tf.uint8)

In [ ]:
y = detector(inputs)
class_ids = y["detection_classes"]

In [ ]:
print(*y.keys(), sep='\n')

In [ ]:
y['num_detections']

In [ ]:
y['detection_classes']

In [ ]:
y['detection_boxes']

In [ ]:
import matplotlib.patches as patches

fig, ax = plt.subplots(1)

ax.imshow(inputs[0].numpy())

# for y['detection_boxes']:
ax.add_patch(patches.Rectangle((50,100),40,30,linewidth=1,edgecolor='r',facecolor='none'))

;